# Examples of Plotting FV3 Forecast Files

FV3 forecasts are written out in two separate files at each output time. Files prefixed `dynf` contain 3D dynamics variables, while files prefixed `phyf` contain physics forecast information, mainly at the surface.

In [ ]:
import math
import os
from argparse import Namespace

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import numpy as np
from netCDF4 import Dataset

In [ ]:
# Provide a file path to a forecast directory. 
# The example below creates a dictionary containing 2 experiments, expt, through the first 4 forecast hours (including 0)


file_path = '/scratch2/BMC/wrfruc/cholt/work/fv3sar_data/chunhua/2019091912.{expt}'

last_fhr = 4
files = {expt: {x: [os.path.join(file_path.format(expt=expt), x + f'{i:03d}.nc') for i in range(5)]  for x in ['dynf', 'phyf']} for expt in ['GSD', 'GFS']}

print('The files dict looks like this:')
files

In [ ]:
# Load each file in the files dict into a NetCDF Dataset

def load_Dataset(files: dict, ds: dict):
    assert(isinstance(files, dict))
    for k, v in files.items():
        if isinstance(v, dict):
            ds[k] = {}
            load_Dataset(v, ds[k])
        else:
            ds[k] = [Dataset(f, 'r') for f in list(v)]

ds_dict = {}
load_Dataset(files, ds_dict)


In [ ]:
# Load the dictionary into a Namespace data structure.
# This step is not necessary, but cuts down the syntax needed to reference each item in the dict.
#
# Example: Retrieve the 0 hr forecast Dataset from GFS Dynamics
#            dict: ds_dict['GFS']['dynf'][0]
#       Namespace: datasets.GFS.dynf[0]


def make_namespace(ns: Namespace(), d: dict):
    assert(isinstance(d, dict))
    for k, v in d.items():
        if isinstance(v, dict):
            leaf_ns = Namespace()
            ns.__dict__[k] = leaf_ns
            make_namespace(leaf_ns, v)
        else:
            ns.__dict__[k] = v
   
    
datasets = Namespace()
make_namespace(datasets, ds_dict)
datasets.GFS.dynf[0]

# Print out the available variables

In [ ]:
print('~~~~~~~~~ DYNAM FILE ~~~~~~~~~~~~~~~')
for v, info in datasets.GFS.dynf[0].variables.items():
    print(v, ':', info.long_name, info.shape)
    
print('~~~~~~~~~ PHYSICS FILE ~~~~~~~~~~~~~~~')
for v, info in datasets.GFS.phyf[0].variables.items():
    print(v, ':', info.long_name, info.shape)

# Plot subplots with experiments and diffs

In [ ]:
def plot_data(dataL, dataR, lat, lon, title, expt):
    
    '''
    Input parameters:
    
        dataL: 2D Numpy array to be plotted in Left column
        dataR: 2D Numpy array to be plotted in Right column
        lat: 2D Numpy array of latitude
        lon: 2D Numpy array of longitude
        title: String describing the variable being plotted.
        
    Draws a Basemap representation with the contoured data overlayed, 
    with a colorbar for each experiment, and the difference between the two.
        
    '''
    
    def trim_grid():
        '''
        The u, v, and H data from analysis are all on grids either one column, or one row smaller than lat/lon. 
        Return the smaller lat, lon grids, given the shape of the data to be plotted.
        Has no effect when all grids are the same size.
        '''
        y, x = np.shape(dataL)
        return lat[:y, :x], lon[:y, :x]
    
    def eq_contours(indata):
        '''
        Returns a balanced set of contours for data that has negative values.
        Also returns default colorbar to use for balanced, vs all positive values.
        '''
        minval = np.amin(indata)
        maxval = np.amax(indata)
        if minval == maxval:
            return np.linspace(-1, 1, 5), 'seismic'
        if np.amin(indata) < 0:
            # Set balanced contours. Choose an odd number in linspace below
            maxval = max(abs(minval), abs(maxval))
            return np.linspace(-maxval, maxval, 21), 'seismic'
        else:
            return np.linspace(minval, maxval, 21), 'hsv'
                     
    
    lat_trim, lon_trim = trim_grid()

    
    fig, ax = plt.subplots(1, 3, figsize=(24, 12))
    
    for i, data in enumerate([dataL, dataR, dataL-dataR]):
        # Check out this link for all cmap options: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
        # A good redwhiteblue cmap for increments is seismic, and for full fields with rainbow, change to hsv
  
        m = Basemap(projection='mill', 
                    llcrnrlon=lon.min()-2,
                    urcrnrlon=lon.max()+2,
                    llcrnrlat=lat.min()-2,
                    urcrnrlat=lat.max()+2,
                    resolution='c',
                    ax=ax[i],
                   )
        x, y = m(lon_trim, lat_trim)
       
        # Use the same contour values for both experiments.
        if i < 2:
            contours, cm = eq_contours(dataL)
        else:
            contours, cm = eq_contours(data)
            
        # Draw the contoured data over the map
        cs = m.contourf(x, y, data, contours, cmap=cm, ax=ax[i])
        m.drawcoastlines();
        m.drawmapboundary();
        m.drawparallels(np.arange(-90.,120.,5),labels=[1,0,0,0]);
        m.drawmeridians(np.arange(-180.,180.,5),labels=[0,0,0,1]);
        fig.colorbar(cs, ax=ax[i], orientation='vertical', shrink=0.25);
        ax[i].set_title(f"{expt[i]}: {title}")

# Plot the variables

In [ ]:
fhr = 0
lat = datasets.GSD.dynf[fhr]['grid_yt'][::] * 180 / math.pi
lon = datasets.GSD.dynf[fhr]['grid_xt'][::] * 180 / math.pi

# List of two experiments, and diff for labeling plots
expt = ['GSD', 'GFS', 'GSD-GFS']

# Variables to plot from dynf and phyf files
vars_ = {
    'dynf': ['ugrd', 'vgrd', 'tmp', 'spfh', 'delz'],
    'phyf': ['tmpsfc', 'soilt1', 'soilt2', 'soilt3', 'albdo_ave',  'gflux', 'sotyp', 'orog', 'hpbl'],
}

# Vertical level (surface is last index, -1; TOA is first index, 0)
lev = -1


file = 'dynf'
for var in vars_[file]:
    dataL = np.squeeze(ds_dict[expt[0]][file][fhr][var][::])[lev]
    dataR = np.squeeze(ds_dict[expt[1]][file][fhr][var][::])[lev]
    title = f'{var} at {fhr} hr fcst'
    plot_data(dataL, dataR, lat, lon, title, expt)

file = 'phyf'
for var in vars_[file]:
    dataL = np.squeeze(ds_dict[expt[0]][file][fhr][var][::])
    dataR = np.squeeze(ds_dict[expt[1]][file][fhr][var][::])
    title = f'{var} at {fhr} hr fcst'
    plot_data(dataL, dataR, lat, lon, title, expt)
        


